# Requirement: rdkit

```bash
conda install rdkit -c rdkit
```

In [1]:
from ipywidgets import HTML

HTML("""
<style>

.smilesApp {
    background-color: white
}
</style>
""")

HTML(value='\n<style>\n\n.smilesApp {\n    background-color: white\n}\n</style>\n')

In [13]:
import nglview as nv
from IPython.display import display
from nglview.widget import Fullscreen
from ipywidgets import Text, Button, AppLayout, VBox, HBox, Output, GridspecLayout, Checkbox, Image
import nglview as nv
from rdkit import Chem # doctest: +SKIP
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from io import BytesIO


class SmilesView(AppLayout):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.view = nv.NGLWidget()
        self.view.gui_style = 'ngl'
        self.view.background = 'black'
        self.text = Text()
        btn = Button(description='Generate')
        btn_fs = Button(description='Fullscreen')
        self.center = self.view
        self._out = Output()
        self._err = Output()
        self._check = Checkbox(description='Replace', value=True)
        bgen = VBox([self.text, self._out])
        bfull = VBox([btn, btn_fs])
        footer = self.footer = HBox([bgen, bfull])
        self.left_sidebar = lbar = Image()
        self.left_sidebar.layout.align_self = 'center'
        lbar.layout.display = 'none'
        self._fs = Fullscreen(self, [self.view])
        self.footer.layout.justify_content = 'center'
        display(self._fs)
        
        @btn.on_click
        def on_click_smiles(_):
            self._handle_smiles_changed()
        
        @btn_fs.on_click
        def on_click_fs(_):
            self._fs.fullscreen()
        
        self.on_displayed(self.display_callback)
        
    def display_callback(self, _):
        s.layout.height = '600px'
        s.view.handle_resize()
            
    def _handle_smiles_changed(self):
        self._out.clear_output()
        self._err.clear_output()
        st = None
        with self._err:
            m = Chem.MolFromSmiles(self.text.value)
            print(type(m))
            m = Chem.AddHs(m)
            print(type(m))
            Chem.AllChem.EmbedMultipleConfs(m, useExpTorsionAnglePrefs=True, useBasicKnowledge=True)
            self.left_sidebar.value = m._repr_png_()
            self.left_sidebar.layout.display = 'flex'
            st = nv.RdkitStructure(m)

        if st:
            if self._check.value:
                if self.view.n_components > 0:
                    for c in self.view:
                        self.view.remove_component(c)
            self.view.add_component(st)
        else:
            self._out.append_stdout("ERROR: bad input")
        
s = SmilesView()
s.add_class('smilesApp')
# smiles example:
# COc1ccc2[C@H](O)[C@@H](COc2c1)N3CCC(O)(CC3)c4ccc(F)cc4
s

Fullscreen()

SmilesView(children=(HBox(children=(VBox(children=(Text(value=''), Output())), VBox(children=(Button(descripti…